In [1]:
import tensorflow as tf
import keras
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

2023-07-08 00:08:01.732884: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 00:08:02.261830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__

'2.12.0'

In [3]:
len(tf.config.list_physical_devices('GPU'))

2023-07-08 00:08:05.305945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-08 00:08:05.322998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-08 00:08:05.323177: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

1

In [11]:
# Create a data generator
class AugmentedDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self,
                 batch_size=32,
                 root_dir=r"/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train",
                 input_shape=(211, 300),
                 shuffle=True,
                labels={'hear': 0,
 'music': 1,
 'bookstore': 2,
 'classroom': 3,
 'doctor': 4,
 'focus': 5,
 'chicken': 6,
 'door': 7,
 'many': 8,
 'polite': 9,
 'good morning': 10,
 'coffee': 11,
 'photographer': 12,
 'hamburger': 13,
 'i': 14,
 'phone': 15,
 'brother': 16,
 'i love you': 17,
 'milk': 18,
 'dog': 19,
 'ocean': 20,
 'research': 21,
 'book': 22,
 'open': 23,
 'money': 24}):
#         
        self.root_dir = root_dir
        print(self.root_dir)
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.labels=labels
        count = 0
        for root_dir, cur_dir, files in os.walk(self.root_dir):
            count += len(files)
        print('file count:', count)
        self.n = count
        self.fileList = self.__create_file_list(self.root_dir, shuffle)
    def __create_file_list(self, root_dir, shuffle):
        count = 0
        fileList = []
        for label in os.listdir(root_dir):
            if label in self.labels:
                for file in os.listdir(os.path.join(root_dir, label)):
                    fileList.append((file, label))
        if shuffle:
            random.shuffle(fileList)
        return fileList
    
    def process_csv_file(self, fileTuple):
        file, label = fileTuple
        filePath = os.path.join(self.root_dir, label, file)
        try:
            csv_data = pd.read_csv(filePath)
        except:
            print("CANT READ CSV: ", filePath)
            return

        if csv_data.isnull().values.any():    
            return False
        try:
            csv_data = csv_data.drop("class", axis = 1)
        except KeyError:
            pass
        return (csv_data.to_numpy(), label)
    
    def set_padding(self, arr, max_length):
        arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
        return np.expand_dims(arr, 0)
    def __get_data(self, filesToProcess):
        X = np.empty((0, self.input_shape[0], self.input_shape[1]))
        Y = np.empty((0,), int)
        for fileTuple in filesToProcess:
            try:
                data, label = self.process_csv_file(fileTuple)
                data = self.set_padding(data, max_length=self.input_shape[0])
                X = np.append(X, data, axis=0)
                Y = np.append(Y, self.labels[label])

            except TypeError:
                pass
#                 print(fileTuple)
        return X, Y
    def on_epoch_end(self):
        pass
    
    def __getitem__(self, index):
        filesToProcess = self.fileList[index * self.batch_size:(index + 1) * self.batch_size]
        batchX, batchY = self.__get_data(filesToProcess)
        return batchX, batchY
    def __len__(self):
        return self.n // self.batch_size
    

In [12]:
def get_labels(folder_path):
    labels = {}
    count = 0
    for folder in os.listdir(folder_path):
            labels[folder] = count
            count += 1
    return labels

labels = get_labels("/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train")
labels

{'hear': 0,
 'music': 1,
 'bookstore': 2,
 'classroom': 3,
 'doctor': 4,
 'focus': 5,
 'chicken': 6,
 'door': 7,
 'many': 8,
 'polite': 9,
 'good morning': 10,
 'coffee': 11,
 'photographer': 12,
 'hamburger': 13,
 'i': 14,
 'phone': 15,
 'brother': 16,
 'i love you': 17,
 'milk': 18,
 'dog': 19,
 'ocean': 20,
 'research': 21,
 'book': 22,
 'open': 23,
 'money': 24}

In [13]:
train_data = AugmentedDataGenerator()
test_data = AugmentedDataGenerator(root_dir=r"/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/test",
                                  batch_size=32)
train_data.fileList[0:5]
test_data.fileList[0:5]

/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train
file count: 42720
/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/test
file count: 9120


[('6q3CLC64igA455_projective_geo_6_False_reflexion_3_.csv', 'i'),
 ('ax2UGtA8h3E2137_projective_geo_12_False_1__2__3_.csv', 'coffee'),
 ('2ofPs_uTel410211_projective_geo_7_False_3_.csv', 'doctor'),
 ('CPAxecqxhzY0_projective_geo_13_True_0_.csv', 'hear'),
 ('afwqCG7GmrQ1632_projective_geo_9_True_reflexion_0__1__2__3_.csv', 'focus')]

In [14]:
# test if the process_csv_file method works
file = "41X2t_s2Ai40.csv"
train_data.process_csv_file((file, "book"))


(array([[ 0.48266143,  0.25151151, -0.66583532, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48195514,  0.25146753, -0.66424268, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48045793,  0.25147596, -0.65664881, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.48019916,  0.24556774, -0.64471287, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48011598,  0.24555989, -0.65619022, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48010522,  0.24555062, -0.66737396, ...,  0.        ,
          0.        ,  0.        ]]),
 'book')

In [15]:
#LSTM model - 25 labels
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(211, 300)))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64, return_sequences=True))
# model_lstm.add(LSTM(64, return_sequences=True))
model_lstm.add(Flatten())
# model_lstm.add(Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.01)))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(Dense(256, activation="relu", ))
model_lstm.add(Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.01)))
# model_lstm.add(Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.05)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(32, activation="relu"))
model_lstm.add(Dense(25, activation='softmax'))

2023-07-08 00:09:29.990635: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 00:09:29.991552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-08 00:09:29.992166: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/Tmp/shariffa/LSTM-25-labelsNewAug.{epoch:02d}-{val_accuracy:.2f}",
#     "/Tmp/linxinle/models/LSTM-25-labels.{epoch:02d}-{val_accuracy:.2f}",

    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [ ]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_lstm.fit(train_data, epochs=50, validation_data=test_data, callbacks=[checkpoint])

Epoch 1/50


2023-07-08 00:10:09.801547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-08 00:10:09.919312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 00:10:09.920072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

1335/1335 [==============================] - ETA: 0s - loss: 5.3770 - accuracy: 0.1472

2023-07-08 00:16:10.283109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-08 00:16:10.411910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 00:16:10.412650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

2023-07-08 00:17:27.725416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 00:17:27.726332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-08 00:17:27.726995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2023-07-08 00:31:32.021377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 00:31:32.022255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-08 00:31:32.022931: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-08 00:31:32.769987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_lstm_2_input' with dtype float and shape [?,211,300]
	 [[{{node serving_default_lstm_2_input}}]]


INFO:tensorflow:Assets written to: /Tmp/shariffa/LSTM-25-labelsNewAug.03-0.30/assets


INFO:tensorflow:Assets written to: /Tmp/shariffa/LSTM-25-labelsNewAug.03-0.30/assets


1335/1335 [==============================] - 424s 318ms/step - loss: 5.1014 - accuracy: 0.4527 - val_loss: 6.7070 - val_accuracy: 0.3013
Epoch 4/50
  78/1335 [>.............................] - ETA: 5:26 - loss: 5.2252 - accuracy: 0.5532

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [82]:
d = {"key1": 10, "key2": 23}
if "key1" in d:
    print("OK")

OK
